# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
import pandas as pd
import numpy as np

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [6]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [10]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    #-2 removes , at end of string
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [3]:
document = ET.parse( './data/mondial_database.xml' )

### 1) 10 Countries with lowest infant mortality rates

In [4]:
#Use dictionary because 2 features to extract
country_di = {}

# Use find and iterfind to extract country and mortality rate from document
for element in document.iterfind('country'):
    country = element.find('name')
    mortality = element.find('infant_mortality')

# Build dictionary with all valid country names and mortality rates
    if country != None and mortality != None:
        country_di[country.text] = float(mortality.text)

# Convert to dataframe, setting country as index
infant_df = pd.DataFrame.from_dict(country_di, orient = 'index')

#Set column to mortality rate and sort
infant_df.columns = ['Infant Mortality Rate']
infant_df.sort_values('Infant Mortality Rate', ascending=True).head(10)

,Infant Mortality Rate
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


### 2) 10 Cities with largest population

In [14]:
# Create list to store city/population
mylist = []

# First find cities and provinces
for country in document.iterfind('country'):
    cities = country.findall('city')
    provinces = country.findall('province')

# Combine province populations
    for province in provinces:
        cities += province.findall('city')

# Fill city names and population by iterating through all cities
    for city in cities:
        city_name = city.find('name').text
        population = city.find('.//population[last()]')
# Convert text attribute to int for all found populations
        if population != None:
            city_pop = int(population.text)
# Add city and population to list
        recent_pop = (city_name, city_pop)
        mylist.append(recent_pop)
# Creates dataframe
labels = ['City', 'Population']
city_df = pd.DataFrame.from_records(mylist, columns = labels)
city_df.sort_values('Population', ascending=False).head(10)

,City,Population
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


### 3) 10 ethnic groups with the largest overall populations

In [41]:
# Dictionary for country/population
country_pop = {}

# Extract country name and population
for element in document.iterfind('country'):
    country = element.find('name')
    population = element.find('.//population[last()]')

# Build dictionary where country and population exist
    if country != None and population != None:
        country_pop[country.text] = int(population.text)

# Creates dataframe
ctry_df = pd.DataFrame.from_dict(country_pop, orient='index').reset_index()
ctry_df.columns = ['Country', 'Population']
ctry_df.head()

,Country,Population
0,Albania,2800138
1,Greece,10816286
2,Macedonia,2059794
3,Serbia,7120666
4,Montenegro,620029


In [42]:
# Blank list for country/ethnicity/percentage to be merged with previous dictionary
ethnicity = []

for country in document.iterfind('country'):
    ctry = country.find('name').text

    for ethnic in country.iterfind('ethnicgroup'):
# Find and append ethnicity and percentage
        ethnic_name = ethnic.text
        ethnic_pct = ethnic.attrib['percentage']
        ethnicity.append([ctry, ethnic_name, ethnic_pct])

# Creates data frame from list and merges it with country popluation dataframe
ethnicity_labels = ['Country', 'Ethnicity', 'Percentage']
ethnicity_df = pd.DataFrame(ethnicity, columns=ethnicity_labels)
ethnicity_df = ethnicity_df.merge(ctry_df, on='Country')
ethnicity_df.head()

,Country,Ethnicity,Percentage,Population
0,Albania,Albanian,95,2800138
1,Albania,Greek,3,2800138
2,Greece,Greek,93,10816286
3,Macedonia,Macedonian,64.2,2059794
4,Macedonia,Albanian,25.2,2059794


In [43]:
# Add column for ethnic population; Percentage*Population = Ethnic Population
ethnicity_df['Ethnic Population'] = (ethnicity_df['Percentage'].astype(float) / 100) * ethnicity_df['Population']
ethnicity_df.head()

,Country,Ethnicity,Percentage,Population,Ethnic Population
0,Albania,Albanian,95,2800138,2.660131e+06
1,Albania,Greek,3,2800138,8.400414e+04
2,Greece,Greek,93,10816286,1.005915e+07
3,Macedonia,Macedonian,64.2,2059794,1.322388e+06
4,Macedonia,Albanian,25.2,2059794,5.190681e+05


In [44]:
# Group by ethnicity to sum all ethnicity populations together and then sort to find top 10
h = ethnicity_df.groupby('Ethnicity').sum()
h['Ethnic Population'].sort_values(ascending = False).head(10)

Ethnicity
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: Ethnic Population, dtype: float64

### 4) a. Longest River

In [51]:
# Dictionary to associate country/code
ctry_dict = {}

for code in document.iterfind('country'):
    ctry_dict[code.attrib['car_code']] = code.find('name').text

# Creates dataframe from dictionary
ctry_codes_df = pd.DataFrame.from_dict(ctry_dict, orient='index').reset_index()
ctry_codes_df.columns = ['Code', 'Country']
ctry_codes_df.head()

,Code,Country
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro


In [72]:
# Creates blank list
river_list = []

# Loops for rivers' names, lengths, and country codes
for rivers in document.iterfind('river'):
    river_name = rivers.find('name').text
    river_length_element = rivers.find('length')
    
    if river_length_element != None:
        river_length = river_length_element.text

# Rivers run though multiple countries; codes must be split after all are selected
        for river_code in rivers.attrib['country'].split():
            river_ctry_code = ctry_dict[river_code]
    
    river_list.append([river_ctry_code, river_code, river_name, float(river_length)])

# Build df from river_list with appropriate labels
River_Labels = ['Country', 'Code', 'River', 'Length']
rivers_df = pd.DataFrame.from_records(river_list, columns=River_Labels)

In [74]:
# Longest River with Country and Name
rivers_df[['Country','Length', 'River']].sort_values(by = 'Length', ascending = False).head(1)

,Country,Length,River
174,Peru,6448.0,Amazonas


### 4) b. Largest Lake

In [78]:
# List for lake names/areas/country codes
lake_list = []

for lakes in document.iterfind('lake'):
    lake_name = lakes.find('name').text
    lake_area_element = lakes.find('area')
    
    if lake_area_element != None:
        lake_area = lake_area_element.text
        
        for lake_code in lakes.attrib['country'].split():
            lake_ctry_code = ctry_dict[lake_code]
    
    lake_list.append([lake_ctry_code, lake_code, lake_name, float(lake_area)])

# Build dataframe from list with labels
Lake_Labels = ['Country', 'Code', 'Lake', 'Area']
lakes_df = pd.DataFrame.from_records(lake_list, columns=Lake_Labels)
lakes_df.head()

,Country,Code,Lake,Area
0,Finland,SF,Inari,1040.0
1,Finland,SF,Oulujaervi,928.0
2,Finland,SF,Kallavesi,472.0
3,Finland,SF,Saimaa,4370.0
4,Finland,SF,Paeijaenne,1118.0


In [81]:
# Largest Lake with Country and Name
lakes_df[['Country','Lake', 'Area']].sort_values(by = 'Area', ascending = False).head(2)

# There is some debate about whether the caspian sea is a lake, thus include 2nd largest lake

,Country,Lake,Area
54,Turkmenistan,Caspian Sea,386400.0
109,United States,Lake Superior,82103.0


### 4) c. Airport at highest elevation

In [89]:
# Creates list for airport names/elevations/ctry codes
airport_list = []

for airport in document.iterfind('airport'):
    
    airport_name = airport.find('name').text
    airport_elev_element = airport.find('elevation')
    
    if airport_elev_element != None:
        airport_elev = airport_elev_element.text
    
        for airport_code in airport.attrib['country'].split():
            airport_ctry_code = ctry_dict[airport_code]
    
    airport_list.append([airport_ctry_code, airport_code, airport_name, airport_elev])

In [96]:
Airport_Labels = ['Country', 'Code', 'Airport', 'Elevation']
airport_text_df = pd.DataFrame.from_records(airport_list, columns=Airport_Labels)

#Convert to numeric in order to sort
airport_df = airport_text_df.apply(pd.to_numeric, errors = 'ignore')
airport_df.head()

,Country,Code,Airport,Elevation
0,Afghanistan,AFG,Herat,977.0
1,Afghanistan,AFG,Kabul Intl,1792.0
2,Albania,AL,Tirana Rinas,38.0
3,Algeria,DZ,Cheikh Larbi Tebessi,811.0
4,Algeria,DZ,Batna Airport,822.0


In [97]:
# Airport at highest elevation
airport_df[['Country','Airport', 'Elevation']].sort_values(by = 'Elevation', ascending = False).head(1)

,Country,Airport,Elevation
80,Bolivia,El Alto Intl,4063.0
